<a href="https://colab.research.google.com/github/AmosNadler/newRepository/blob/master/trader_classification_active_management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving trader_data_df_1.csv to trader_data_df_1.csv


In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('trader_data_df_1.csv')
df1 = df.rename(columns={'Unnamed: 0': 'idx', 'Investor #': 'Investor'})

In [ ]:
df1

,idx,Transaction Date,Transaction Type,SYMBOL,Quantity,Price,Amount,Investor
0,0,29/09/2020,Bought,AAPL,4.000000,113.360878,453.443512,0
1,1,29/09/2020,Bought,ACAD,4.000000,40.410000,161.639999,0
2,2,29/09/2020,Bought,AMD,4.000000,81.769997,327.079987,0
3,3,29/09/2020,Bought,AMWL,4.000000,28.100000,112.400001,0
4,4,29/09/2020,Bought,EBS,4.000000,104.339996,417.359985,0
...,...,...,...,...,...,...,...,...
739,34,19/08/2021,Bought,ACAD,6.146560,16.139999,99.205475,6
740,35,08/09/2021,Bought,TREE,2.352000,159.119995,374.250228,6
741,36,22/09/2021,Bought,EBS,2.352000,53.849998,126.655196,6
742,37,28/09/2021,Bought,AMWL,4879.230235,9.230000,45035.292840,6


In [ ]:
df2 = df1.sort_values(['Investor', 'Transaction Date', 'SYMBOL'])
new_index = range(0, len(df2))
df2 = df2.reindex(new_index)
df2

,idx,Transaction Date,Transaction Type,SYMBOL,Quantity,Price,Amount,Investor
0,0,29/09/2020,Bought,AAPL,4.000000,113.360878,453.443512,0
1,1,29/09/2020,Bought,ACAD,4.000000,40.410000,161.639999,0
2,2,29/09/2020,Bought,AMD,4.000000,81.769997,327.079987,0
3,3,29/09/2020,Bought,AMWL,4.000000,28.100000,112.400001,0
4,4,29/09/2020,Bought,EBS,4.000000,104.339996,417.359985,0
...,...,...,...,...,...,...,...,...
739,34,19/08/2021,Bought,ACAD,6.146560,16.139999,99.205475,6
740,35,08/09/2021,Bought,TREE,2.352000,159.119995,374.250228,6
741,36,22/09/2021,Bought,EBS,2.352000,53.849998,126.655196,6
742,37,28/09/2021,Bought,AMWL,4879.230235,9.230000,45035.292840,6


In [ ]:
investor_list = df2['Investor'].unique()
for trader in investor_list:
    symbol_list = df2.loc[df2['Investor'] == trader, 'SYMBOL'].unique()
    for symbol in symbol_list:
        df2.loc[(df2['Investor'] == trader) & (df2['SYMBOL'] == symbol), 'Cumulated_Holding'] = df2.loc[(df2['Investor'] == trader) & (df2['SYMBOL'] == symbol), 'Quantity'].cumsum(axis=0)
        df2.loc[(df2['Investor'] == trader) & (df2['SYMBOL'] == symbol), 'Cumulated_Spent'] = df2.loc[(df2['Investor'] == trader) & (df2['SYMBOL'] == symbol), 'Amount'].cumsum(axis=0)
        df2.loc[(df2['Investor'] == trader) & (df2['SYMBOL'] == symbol), 'Avg_Price'] = df2.loc[(df2['Investor'] == trader) & (df2['SYMBOL'] == symbol), 'Quantity'].cumsum(axis=0)
df2.loc[:, 'Avg_Price'] = df2['Cumulated_Spent']/df2['Cumulated_Holding']
df2['Symbol_C_P&L'] = ((df2['Price'] - df2['Avg_Price'])*df2['Cumulated_Holding']).round(6)
df2['Symbol_C_P&L%'] = ((df2['Symbol_C_P&L']/df2['Cumulated_Spent'])*100).round(6)

Example:

In [ ]:
df2[(df2['Investor'] == 0) & (df2['SYMBOL'] == 'AAPL')]

,idx,Transaction Date,Transaction Type,SYMBOL,Quantity,Price,Amount,Investor,Cumulated_Holding,Cumulated_Spent,Avg_Price,Symbol_C_P&L,Symbol_C_P&L%
0,0,29/09/2020,Bought,AAPL,4.00,113.360878,453.443512,0,4.00,453.443512,113.360878,0.000000,0.000000
54,54,30/10/2020,Bought,AAPL,0.40,108.164307,43.265723,0,4.40,496.709235,112.888462,-20.786286,-4.184800
62,62,02/11/2020,Bought,AAPL,0.44,108.074883,47.552948,0,4.84,544.262183,112.450864,-21.179752,-3.891461


In [ ]:
portfolio = {}
for trader in investor_list:
    symbol_list = df2.loc[df2['Investor'] == trader, 'SYMBOL'].unique()
    spent = []
    PL = []
    for symbol in symbol_list:
        latest_PL = df2.loc[(df2['Investor'] == trader) & (df2['SYMBOL'] == symbol), 'Symbol_C_P&L'].iloc[-1]
        latest_spent = df2.loc[(df2['Investor'] == trader) & (df2['SYMBOL'] == symbol), 'Cumulated_Spent'].iloc[-1]
        spent.append(latest_spent)
        PL.append(latest_PL)
    total_spent = sum(spent).round(2)
    total_PL = sum(PL).round(2)
    PL_percent = (100*total_PL/total_spent).round(4)
    portfolio[trader] = [trader, total_spent, total_PL, PL_percent]

In [ ]:
overview = pd.DataFrame.from_dict(portfolio, orient='index', columns=['Trader', 'Spent', 'P&L', 'P&L_%'])
overview.sort_values(['P&L_%'], ascending=False)

,Trader,Spent,P&L,P&L_%
4,4,411203.30,-11754.94,-2.8587
0,0,23767.04,-1416.69,-5.9607
1,1,229972.72,-20248.80,-8.8049
2,2,679870.72,-64138.32,-9.4339
3,3,1149608.54,-189096.34,-16.4488
5,5,1958241.27,-496429.77,-25.3508
6,6,226531.50,-62916.39,-27.7738
